# Lecture 1 - 31.05.2021 

# Exercise: Range compression

In [18]:
# --- To get started, we set the ipympl backend, which makes matplotlib plots interactive.
# --- We do this using a magic command, starting with %.
%matplotlib widget

# --- import libraries

import sys
sys.path.append('/projects/src/')


import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq, fftshift, ifft

# --- import rat class
from ste_io import *


## Exercise 1

The typical P-band chirp transmitted by the F-SAR system during the AfriSAR campaign is characterized by the following parameters:

- Chirp duration: 1.0016000e-05 s,
- Chirp bandwidth: 50 MHz.

Further, let's assume a range spacing (= range sampling) equal to 1.1988876 m and 3772 range pixels (= samples). 

1. Which is the expected resolution?
2. Generate the chirp signal and calculate its spectrum;
3. Plot signal and spectrum.

Tips:
1. One can work equivalently in the time or the range domain (and the related frequency domains). Just remember that 
$$range = \frac{c \cdot time}{2}$$
where c is the speed of light: c=2.9972190e+08 m/s.


### Solution

In [19]:
# --- create useful functions

def rect(t, T0):
    # Implements y = rect(t/T0) = 1 if abs(t/T0) <= 0.5, 0 otherwise 
    # All input / outputs are Numpy arrays
    
    N = t.shape[0]
    y = np.zeros(N, 'float32') 
    y[np.abs(t/T0) <= 0.5] = 1
    
    return y

# --- Input parameters

# number of samples 
Ns = 3772
# chirp duration, s
Tc = 1.0016000e-05
# chirp bandwidth, Hz
BW = 50.0e6
# range sampling, m
rs = 1.1988876
# speed of light
c0 = 2.9972190e+08

In [31]:
# --- Derived parameters (calculate following parameters)

# range resolution
rres = c0 / 2 / BW
# chirp rate
k = BW / Tc
# signal duration
Td = (Ns - 1)*rs*2/c0
# sampling in time
Ts = rs *2/c0

# max duration for zoom plotting, s
Tzoom = 2e-5

# Print answer to Question 1...
print('The range resolution is ' + str(rres) + ' m.')

# time axis
time = np.linspace(-Td/2, Td/2, Ns)
print(time[0])
print(time[Ns-1])

# chirp generation
chirp = np.exp(1j*np.pi*k*(time)**2) * rect(time, Tc)

plt.figure()
plt.plot(time, np.real(chirp), lw = 3)

# spectrum
frequency = np.linspace(-1/Ts/2, 1/Ts/2, Ns)/1e6
print(frequency[0]) 
print(frequency[Ns-1]) 
chirp_spectrum = fftshift(fft(chirp))

plt.figure()
plt.plot(frequency, np.abs(chirp_spectrum))

The range resolution is 2.997219 m.
-1.5084e-05
1.5084e-05


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-62.50000000000001
62.50000000000001


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Exercise 2

The files *./data/01-sar/signal1_rc.npy* and *./data/01-sar/signal2_rc.npy* contain the received signals along two range lines originated by the same range chirp and sampled with the same parameters as in Exercise 1.

1. Open the files (use np.load);
2. Plot the real part of the signals;
3. Focus in range by using a matched filter;
4. ... how many scatterers do you see? and at which range?

### Solution

In [41]:

# generate impulse response of matched filter
matched_filter = np.conj(chirp[::-1])

# open signal 1
signal1 = np.load('./data/01-sar/signal1_rc.npy') 

# focus
output1 = np.convolve(signal1, matched_filter, 'same')

# plot
plt.figure()
plt.plot(time, np.real(signal1))
plt.figure()
plt.plot(time, np.real(output1))

# open signal 2
signal2 = np.load('./data/01-sar/signal2_rc.npy')

# focus
output2 = np.convolve(signal2, matched_filter, 'same')

# plot
plt.figure()
plt.plot(time, np.real(signal2))
plt.figure()
plt.plot(time*c0/2, np.real(output2))

# spectrum of filter response
signal2_spectrum = fft(signal2)
matched_filter_spectrum = fft(matched_filter)
output2_spectrum = matched_filter_spectrum * signal2_spectrum
output2_2 = fftshift(ifft(output2_spectrum))

plt.figure()
plt.plot(time,np.real(output2_2))







Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Exercise 3

We are ready now to focus real F-SAR P-band data !

1. Open the raw data image in the file *./data/01-sar/raw-img.rat*, and visualize it (only the amplitude).
2. Compress it in range using the same chirp as in the Exercise 1. What do you see?

Tips:
Use the provided rat class to open the data / or use open_rasterio from xarray simliar to notebook: *00-read-rat-file.ipynb*.

### Solution

In [46]:
im = rrat('./data/01-sar/raw-img.rat')

print(im.shape)
im.dtype

(9984, 3772)


dtype('complex64')

In [47]:
plt.figure()
plt.imshow(np.abs(im), vmin = 0, vmax = np.mean(abs(im)), cmap = 'gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …